In [2]:
import numpy as np

bw = 4

In [23]:
class Equiangulartovtk:
     
    def make_coords(self, bw):
        alpha = np.arange(2 * bw) * np.pi / bw
        beta = np.arange(2 * bw) * np.pi / (2. * bw)
      
        theta, phi = np.meshgrid(*(beta, alpha),indexing='ij')
        ct = np.cos(theta).flatten()
        st = np.sin(theta).flatten()
        cp = np.cos(phi).flatten()
        sp = np.sin(phi).flatten()
        x = st * cp
        y = st * sp
        z = ct
        coords = np.vstack([x, y, z]).T
        coords = np.asarray(coords, dtype=np.float32)
        return coords[2*bw-1:]

    def __init__(self, bw=4):
        from pyevtk.hl import unstructuredGridToVTK
        from pyevtk.vtk import VtkTriangle, VtkQuad
        
        self.coords = self.make_coords(bw)
        self.x = self.coords[:,0]
        self.y = self.coords[:,1]
        self.z = self.coords[:,2]
        width = 2*bw
        
        
        tri = []
        # north pole
        for i in range(width):
            tri.append([0, i+1, (i+1)%width+1])
        
        # inner layers
        for j in range(width-2):
            s = j*width  # s stands for "shift"
            for i in range(width):
                tri.append([s + (i+1), s + (i+1)%width+1, s + (i+1)%width+1 + width])
            for i in range(width):
                tri.append([s + (i+1), s + (i+1) + width, s + (i+1)%width + width + 1])
                
       
        npix = width*(width-1)+1
        last_row = np.arange(npix-width, npix)
        start = last_row[0]
        for i in last_row[:-2]:
            tri.append([start, i+1, i+2])
        
        
        self.simplices = np.asarray(tri)
        self.conn = self.simplices.flatten()
        self.offset = np.arange(3,len(self.conn)+1, 3)
        self.ctype = np.ones(len(self.offset))*VtkTriangle.tid

    def save(self, title='unstructured', pointData=None):
        from pyevtk.hl import unstructuredGridToVTK
        if pointData is not None:
            unstructuredGridToVTK(
                title,
                self.x,
                self.y,
                self.z,
                connectivity=self.conn,
                offsets=self.offset,
                cell_types=self.ctype,
                pointData={'Temp': pointData})
        else:
            unstructuredGridToVTK(
                title,
                self.x,
                self.y,
                self.z,
                connectivity=self.conn,
                offsets=self.offset,
                cell_types=self.ctype,
                pointData={'Temp': self.x})

In [24]:
e = Equiangulartovtk(8)

In [17]:
e.save(pointData=e.z)


In [29]:
import meshio

cells = {"triangle": e.simplices}
meshio.write_points_cells(
    "equiangular/normal/meshes/manual_equi_{}.xml".format(bw),
    e.coords,
    cells,
)